In [ ]:
import json
import optuna
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Data loading

In [ ]:
# CHANGE ME
dataset_path = "../Dataset/Regional arrival times/regional_arrival_times_1.json"
feature_name = "arrival_times"

In [ ]:
naca_numbers = ['maximum_camber', 'maximum_camber_position', 'maximum_thickness']

dataset = []
with open(dataset_path, 'r') as dataset_file:
  samples = json.load(dataset_file)
  for sample in samples:
    dataset.append({
        "features": sample["features"][feature_name],
        "labels": list(sample["naca_numbers"].values())
    })

### Shuffling the dataset

In [ ]:
# Shuffling the dataset
np.random.shuffle(dataset)

### Training and test set

In [ ]:
# Defining the training and test set splitting percentage
split_percentage = 0.8

# Computing the number of training samples according to the splitting percentage
num_training_samples = int(np.floor(split_percentage * len(dataset)))

# Extracting the training and test set
training_set, test_set = dataset[:num_training_samples], dataset[num_training_samples:]

In [ ]:
# Extracting the training features and labels
train_features = np.array([sample["features"] for sample in training_set])
train_labels = np.array([sample["labels"] for sample in training_set])

# Extracting the test features and labels
test_features = np.array([sample["features"] for sample in test_set])
test_labels = np.array([sample["labels"] for sample in test_set])

### Data normalization

In [ ]:
# Computing the mean and standard deviation of the training features
mean = train_features.mean(axis=0)
std = train_features.std(axis=0)

In [ ]:
# Normalizing the training and test features w.r.t. the training statistics
normalized_train_features = (train_features - mean) / std
normalized_test_features = (test_features - mean) / std

### Creating the study cases

In [ ]:
epochs = 100 # Number of training epochs
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) # Early stopping with a patience of 10 epochs

In [ ]:
# Function to create a testing model
def createModel(trial):
    # Creating the Model
    model = keras.Sequential()

    # Dropout rate
    dropout_rate = trial.suggest_discrete_uniform("dropout_rate", 0.01, 0.3, 0.01)

    # Input layer
    model.add(keras.layers.InputLayer(input_shape=[np.shape(normalized_train_features)[1]]))
    
    # Number of hidden layers
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
    
    # Iterating over the hidden layers
    for i in range(num_hidden_layers):
        # Number of hidden units
        num_units = trial.suggest_categorical(f"num_units__hidden_layer_{i+1}", [16*j for j in range(1, 17)])

        # Adding the hidden layer
        model.add(keras.layers.Dense(num_units, activation=tf.nn.relu))
        model.add(keras.layers.Dropout(rate=dropout_rate))

    # Output layer
    model.add(keras.layers.Dense(len(naca_numbers)))

    # Compiling the model
    model.compile(loss='mse', optimizer="adam", metrics=['mae'])
    
    return model
    

In [ ]:
# Function to train the model
def train(trial, model):
    model.fit(
        normalized_train_features, 
        train_labels,
        epochs=epochs,
        validation_split=0.2,
        verbose=0,
        callbacks=[early_stopping, optuna.integration.TFKerasPruningCallback(trial, 'val_mae')]
    )

In [ ]:
# Function to evaluate the model
def evaluate(model):
    loss, mae = model.evaluate(normalized_test_features, test_labels, verbose=0)
    return loss, mae

In [ ]:
# Objective function to be minimized
def objective(trial):
    # Building the model
    model = createModel(trial)

    # Training the model
    train(trial, model)

    # Evaluating the model
    _, mae = evaluate(model)

    return mae

### Evaluating the hyperparameters

In [ ]:
# Creating the study object with the specified configurations
study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=20)
)

# Running the study
study.optimize(objective, n_trials=100)

In [ ]:
# Extractig the pruned and complete trials
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]

# Displaying the study statistics
print("STUDY STATISTICS")
print(f"Number of finished trials --> {len(study.trials)}")
print(f"Number of pruned trials --> {len(pruned_trials)}")
print(f"Number of complete trials --> {len(complete_trials)}")

In [ ]:
# Extracting the best trial from the study performed
trial = study.best_trial

# Displaying the obtained results
print("BEST TRIAL")
print(f"Mean Absolute Error --> {trial.value}\n")

print("BEST HYPERPARAMETERS")
for key, value in trial.params.items():
    print(f"{key}: {value}")